# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format, to_timestamp, dayofyear, dayofweek
from pyspark.sql import SQLContext 
from pyspark.sql import functions as F
from io import StringIO, BytesIO
import sas_reader

In [2]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.6,com.amazonaws:aws-java-sdk:1.7.4,net.java.dev.jets3t:jets3t:0.9.4") \
        .getOrCreate()  

In [3]:
sqlContext = SQLContext(spark)

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

# Immigration data

## Immigration Reference Data

Parse the file I94_SAS_Labels_Descriptions.SAS and extract the descriptive values for the columns in sas_data.
The file is parsed by sas_reader.py 

### 1. i94addr - State code 

In [4]:
state_codes = sas_reader.proc_sasref('i94addr')

In [5]:
# Convert Dictionary into Pandas Dataframe
state_codes_Pdf=pd.DataFrame.from_dict(state_codes, orient='index',columns=['State'])

In [6]:
# Critical to add to dataframe to preserve index in transform to Spark Dataframe
state_codes_Pdf.reset_index(drop=False, inplace=True)

In [7]:
state_codes_Pdf.head()

,index,State
0,AL,ALABAMA
1,AK,ALASKA
2,AZ,ARIZONA
3,AR,ARKANSAS
4,CA,CALIFORNIA


In [8]:
# Create Spark dataframe
state_codes_df = spark.createDataFrame(state_codes_Pdf)

### 2. i94port - Port Codes 

In [9]:
# 1. Import dictionary
port_codes = sas_reader.proc_sasref('i94port')
# 2. Create pandas Dataframe
port_codes_Pdf=pd.DataFrame.from_dict(port_codes, orient='index',columns=['Port'])
port_codes_Pdf.reset_index(drop=False, inplace=True)
# 3. Convert to spark Dataframe
port_codes_df = spark.createDataFrame(port_codes_Pdf)

In [10]:
port_codes_df.show()

+-----+--------------------+
|index|                Port|
+-----+--------------------+
|  ALC|            ALCAN AK|
|  ANC|        ANCHORAGE AK|
|  BAR|BAKER AAF - BAKER...|
|  DAC|    DALTONS CACHE AK|
|  PIZ|DEW STATION PT LA...|
|  DTH|     DUTCH HARBOR AK|
|  EGL|            EAGLE AK|
|  FRB|        FAIRBANKS AK|
|  HOM|            HOMER AK|
|  HYD|            HYDER AK|
|  JUN|           JUNEAU AK|
|  5KE|        KETCHIKAN AK|
|  KET|        KETCHIKAN AK|
|  MOS|MOSES POINT INTER...|
|  NIK|          NIKISKI AK|
|  NOM|              NOM AK|
|  PKC|      POKER CREEK AK|
|  ORI|   PORT LIONS SPB AK|
|  SKA|          SKAGWAY AK|
|  SNP|   ST PAUL ISLAND AK|
+-----+--------------------+
only showing top 20 rows



### 3. Transport mode Codes - i94mode

In [14]:
# 1. Import dictionary
mode_codes = sas_reader.proc_sasref('i94mode')
# 2. Create pandas Dataframe
mode_codes_Pdf=pd.DataFrame.from_dict(mode_codes, orient='index',columns=['Mode'])
mode_codes_Pdf.reset_index(drop=False, inplace=True)
# 3. Convert to spark Dataframe
mode_codes_df = spark.createDataFrame(mode_codes_Pdf)

In [15]:
mode_codes_df.printSchema()

root
 |-- index: string (nullable = true)
 |-- Mode: string (nullable = true)



## Immigration Dataset - sas_data

In [11]:
immig_df=spark.read.parquet('sas_data') 

In [12]:
immig_df.show()

+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|    cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|5748517.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     CA|20582.0|  40.0|    1.0|  1.0|20160430|     SYD| null|      G|      O|   null|      M| 1976.0|10292016|     F|  null|     QF|9.495387003E10|00011|      B1|
|5748518.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     NV|20591.0|  32.0|    1.0|  

## Build Denormalized Immigration Fact

In [16]:
immig_fact_df = immig_df \
        .join(port_codes_df, immig_df.i94port == port_codes_df.index, how='left')\
        .join(state_codes_df, immig_df.i94addr == state_codes_df.index)\
        .join(mode_codes_df, immig_df.i94mode == mode_codes_df.index)\
        .select(immig_df.cicid, immig_df.i94addr, state_codes_df.State, immig_df.i94port, port_codes_df.Port, immig_df.i94mode, mode_codes_df.Mode)

In [17]:
immig_fact_df.show(5)

+---------+-------+-------+-------+-------------+-------+----+
|    cicid|i94addr|  State|i94port|         Port|i94mode|Mode|
+---------+-------+-------+-------+-------------+-------+----+
|5833042.0|     AZ|ARIZONA|    FMY|FORT MYERS FL|    1.0| Air|
|3691499.0|     HI| HAWAII|    HHW|  HONOLULU HI|    1.0| Air|
|5833464.0|     AZ|ARIZONA|    FMY|FORT MYERS FL|    1.0| Air|
|3691500.0|     HI| HAWAII|    HHW|  HONOLULU HI|    1.0| Air|
|5833466.0|     AZ|ARIZONA|    FMY|FORT MYERS FL|    1.0| Air|
+---------+-------+-------+-------+-------------+-------+----+
only showing top 5 rows



### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here





### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.